#### Three Year Avg for Players

In [26]:
# packages and file import
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import sys
function_dir = 'C:/Users/dalto/OneDrive/Pictures/Documents/Projects/MLB Season Model/tools'
sys.path.append(function_dir)
os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Projects/MLB Season Model/')

import data

In [33]:
data_02_24 = pd.read_csv('./data/cleaned/data_02_24.csv')
data_15_24 = pd.read_csv('./data/cleaned/data_15_24.csv')
data_23_24 = pd.read_csv('./data/cleaned/data_23_24.csv')

In [34]:
data_02_24 = data_02_24.sort_values(by=['Name', 'Season'], ascending=[True, False])
data_02_24 = data_02_24.groupby('Name').head(4).reset_index(drop=True)
data_02_24 = data_02_24.drop(columns=['Unnamed: 0'])

In [35]:
data_15_24 = data_15_24.sort_values(by=['Name', 'Season'], ascending=[True, False])
data_15_24 = data_15_24.groupby('Name').head(4).reset_index(drop=True)
data_15_24 = data_15_24.drop(columns=['Unnamed: 0'])

normalize data to deal with 2020

In [36]:
from tools.zscore_pa import normalize
norm_02_24 = normalize(data_02_24)
norm_15_24 = normalize(data_15_24)

take three year weighted avg

In [37]:
def three_year_avg(df):
    combined_rows = []
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    numeric_cols = [col for col in numeric_cols if col not in ['Season', 'Age', 'PA']]  # exclude Season, Age, PA

    for name, group in df.groupby('Name'):
        group_sorted = group.sort_values('Season')
        n = len(group_sorted)
        if n < 1:
            continue
        for i in range(n):
            window = group_sorted.iloc[i:i+3]
            if len(window) == 0:
                continue
            # Use the latest season and age in the window
            season_label = window.iloc[-1]['Season']
            age_label = window.iloc[-1]['Age']
            pa_label = window['PA'].mean()
            # Average numeric columns (except Season, Age, PA)
            averaged = window[numeric_cols].multiply(window['PA'], axis=0).sum() / window['PA'].sum()
            new_row = {'Name': name, 'Season': season_label, 'Age': age_label, 'PA': pa_label}
            for col in numeric_cols:
                new_row[col] = averaged[col]
            combined_rows.append(new_row)
            if i + 3 >= n:
                break

    avg_three = pd.DataFrame(combined_rows)
    return avg_three


In [38]:
weighted_02_24 = three_year_avg(data_02_24)
weighted_15_24 = three_year_avg(data_15_24)

#### Export to CSV

In [ ]:
weighted_02_24.to_csv('./data/multi_season/weighted_02_24.csv')
weighted_15_24.to_csv('./data/multi_season/weighted_15_24.csv')